<a href="https://colab.research.google.com/github/anannayajannat/HybridQCNN-Model/blob/main/2Augprob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip --quiet

In [ ]:
!pip install pennylane==0.21.0 tensorflow==2.12.0 --quiet
!pip install -U tensorflow_datasets --quiet

In [ ]:
!pip install numpy==1.23.5 tensorflow==2.12.0  pennylane==0.21.0 --quiet

!pip install tensorflow_datasets --quiet
!pip install scikit-learn keras-tuner matplotlib --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.3.2 which is incompatible.
orbax-checkpoint 0.11.21 requires jax>=0.5.0, but you have jax 0.4.30 which is incompatible.
keras-hub 0.21.1 requires keras>=3.5, but you have keras 2.12.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.12.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.12.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tenso

In [ ]:
!pip uninstall -y autograd
!pip install autograd==1.8.0 --quiet

Found existing installation: autograd 1.8.0
Uninstalling autograd-1.8.0:
  Successfully uninstalled autograd-1.8.0


In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [ ]:
import pennylane as qml
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import math

from pennylane.qnn.tensorflow import KerasLayer
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
n_qubits = 4   # Quantum circuit will use 4 qubits
dev = qml.device("default.qubit", wires=n_qubits)

#Define Quantum Circuit
@qml.qnode(dev, interface="tf")
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    # Return expectation values directly — no TF ops here
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


#Wrap Quantum Circuit as Keras Layer
weight_shapes = {"weights": (2, n_qubits)} #trainable parameters per layer

quantum_layer = qml.qnn.KerasLayer(
    quantum_circuit,
    weight_shapes=weight_shapes,
    output_dim=n_qubits
)

#Hybrid Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(n_qubits),  # Reduce to quantum input dim (4)
    tf.keras.layers.Activation('tanh'),
    quantum_layer,  #  Quantum block

    # Add Lambda layer to cast output to float32
    # & take real part (avoid complex warning)
    tf.keras.layers.Lambda(lambda x: tf.cast(tf.math.real(x), tf.float32)),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


AttributeError: module 'pennylane.qnn' has no attribute 'KerasLayer'

In [ ]:
# Load Fashion MNIST dataset
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

class_names = metadata.features['label'].names
print("Class names:", class_names)

num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print(f"Number of training examples: {num_train_examples}")
print(f"Number of test examples:     {num_test_examples}")


def normalize(images, labels):
    images = tf.cast(images, tf.float32) / 255.0
    return images, labels


train_dataset = train_dataset.map(normalize).cache().repeat().shuffle(num_train_examples).batch(128)
test_dataset = test_dataset.map(normalize).cache().batch(128)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Limit steps_per_epoch to 500 for faster training
# model.fit(train_dataset, epochs=10, steps_per_epoch=num_train_examples/64)
model.fit(train_dataset, epochs=10, steps_per_epoch=100)
loss, acc = model.evaluate(test_dataset)
print(f"Restored model accuracy: {acc:.4f}")
new_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(n_qubits),  # Reduce to quantum input dim (4)
    tf.keras.layers.Activation('tanh'),  # Normalize values in [-1, 1]

    # Remove this layer: tf.keras.layers.Lambda(lambda x: tf.cast(x, tf.float32)),

    quantum_layer,  # <-- Quantum block

    # Add Lambda layer to cast output to float32 and take real part (avoid complex warning)
    tf.keras.layers.Lambda(lambda x: tf.cast(tf.math.real(x), tf.float32)),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

new_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# new_model.load_weights("abc.h5")
new_model.load_weights('hybrid_20_epochs.h5')
loss, acc = new_model.evaluate(test_dataset)
print(f"Restored new model accuracy: {acc:.4f}")
new_model.fit(train_dataset, initial_epoch=20, epochs=25, steps_per_epoch=100)
loss, acc = new_model.evaluate(test_dataset)
print(f"Restored new model accuracy: {acc:.4f}")
